In [1]:
import pandas as pd
sms_spam = pd.read_csv("SMSSpamCollection", sep = "\t", header =None, names = ["Label", "SMS"])

print(sms_spam.shape)
sms_spam.head()

(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Below, we see that about 87% of the messages are ham, and the remaining 13% are spam. This sample looks representative, since in practice most messages that people receive are ham.

In [2]:
sms_spam["Label"].value_counts(normalize = True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## Training and Test Set

We're now going to split our dataset into a training and a test set, where the training set accounts for 80% of the data, and the test set for the remaining 20%.

In [4]:
data_randomized = sms_spam.sample(frac = 1, random_state = 1)

training_test_index = round(len(data_randomized) * 0.8)

training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

training_set.shape, test_set.shape

((4458, 2), (1114, 2))

We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

In [5]:
training_set["Label"].value_counts(normalize=True)

ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [6]:
test_set["Label"].value_counts(normalize=True)

ham     0.868043
spam    0.131957
Name: Label, dtype: float64

The results look good! We'll now move on to cleaning the dataset.

## Data Cleaning

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

Essentially, we want to bring data to this format:

![img](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)


### Letter Case and Punctuation

We'll begin with removing all the punctuation and bringing every letter to lower case.

In [7]:
training_set["SMS"] = training_set["SMS"].str.replace("\W"," ").str.lower()
training_set.head()

C:\Users\Lenovo T 470\AppData\Local\Temp\ipykernel_6772\3506881641.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  training_set["SMS"] = training_set["SMS"].str.replace("\W"," ").str.lower()


,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating the Vocabulary

Let's now move to creating the vocabulary, which in this context means a list with all the unique words in our training set.

In [8]:
training_set["SMS"] = training_set["SMS"].str.split()

training_set["SMS"]

0                       [yep, by, the, pretty, sculpture]
1       [yes, princess, are, you, going, to, make, me,...
2                         [welp, apparently, he, retired]
3                                                [havent]
4       [i, forgot, 2, ask, ü, all, smth, there, s, a,...
                              ...                        
4453    [sorry, i, ll, call, later, in, meeting, any, ...
4454    [babe, i, fucking, love, you, too, you, know, ...
4455    [u, ve, been, selected, to, stay, in, 1, of, 2...
4456    [hello, my, boytoy, geeee, i, miss, you, alrea...
4457                              [wherre, s, my, boytoy]
Name: SMS, Length: 4458, dtype: object

In [9]:
vocabulary = []

for sms in training_set["SMS"]:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))        

It looks like there are 7,783 unique words in all the messages of our training set.

In [10]:
len(vocabulary)

7783

### The Final Training Set

We're now going to use the vocabulary we just created to make the data transformation we want.

In [11]:
pd.DataFrame({"yep": [0] * len(training_set["SMS"]), "to": [0] * len(training_set["SMS"])})

,yep,to
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
4453,0,0
4454,0,0
4455,0,0
4456,0,0


In [12]:
training_set["SMS"].head()

0                    [yep, by, the, pretty, sculpture]
1    [yes, princess, are, you, going, to, make, me,...
2                      [welp, apparently, he, retired]
3                                             [havent]
4    [i, forgot, 2, ask, ü, all, smth, there, s, a,...
Name: SMS, dtype: object

In [13]:
word_counts_per_sms = {unique_word : [0] * len(training_set["SMS"]) for unique_word in vocabulary}

for index, sms in enumerate(training_set["SMS"]):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [16]:
word_counts_per_sms["yes"][1]

1

In [17]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,global,africa,muz,landlineonly,sorrow,people,dialling,networking,antelope,bros,...,psychic,invest,snap,4mths,pence,reception,needed,fa,abj,4ward
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
word_counts["prize"].sum()

76

In [19]:
training_set_clean = pd.concat([training_set, word_counts], axis = 1)
training_set_clean.head()

,Label,SMS,global,africa,muz,landlineonly,sorrow,people,dialling,networking,...,psychic,invest,snap,4mths,pence,reception,needed,fa,abj,4ward
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

We're now done with cleaning the training set, and we can begin creating the spam filter. The Naive Bayes algorithm will need to answer these two probability questions to be able to classify new messages:

\begin{equation}
P(Spam | w_1,w_2, ..., w_n) \propto P(Spam) \cdot \prod_{i=1}^{n}P(w_i|Spam)
\end{equation}

\begin{equation}
P(Ham | w_1,w_2, ..., w_n) \propto P(Ham) \cdot \prod_{i=1}^{n}P(w_i|Ham)
\end{equation}


Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, we'll need to use these equations:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}


Some of the terms in the four equations above will have the same value for every new message. We can calculate the value of these terms once and avoid doing the computations again when a new messages comes in. Below, we'll use our training set to calculate:

- P(Spam) and P(Ham)
- N<sub>Spam</sub>, N<sub>Ham</sub>, N<sub>Vocabulary</sub>

We'll also use Laplace smoothing and set $\alpha = 1$.

In [22]:
spam_message = training_set_clean[training_set_clean["Label"] == "spam"]
ham_message = training_set_clean[training_set_clean["Label"] == "ham"]

p_spam = len(spam_message)/len(training_set_clean)
p_ham = len(ham_message)/len(training_set_clean)

n_word_per_spam_message = spam_message["SMS"].apply(len)
n_spam = n_word_per_spam_message.sum()

n_word_per_ham_message = ham_message["SMS"].apply(len)
n_ham = n_word_per_ham_message.sum()

n_vocabulary = len(vocabulary)

alpha = 1

## Calculating Parameters

Now that we have the constant terms calculated above, we can move on with calculating the parameters $P(w_i|Spam)$ and $P(w_i|Ham)$. Each parameter will thus be a conditional probability value associated with each word in the vocabulary.

The parameters are calculated using the formulas:

\begin{equation}
P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}
\end{equation}

\begin{equation}
P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}
\end{equation}

In [23]:
spam_message["yes"].sum()

16

In [26]:
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}


for word in vocabulary:
    n_word_given_spam = spam_message[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha)/(n_spam + alpha * n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    
    n_word_given_ham = ham_message[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha)/(n_ham + alpha * n_vocabulary)
    parameters_ham[word] = p_word_given_ham

## Classifying A New Message

Now that we have all our parameters calculated, we can start creating the spam filter. The spam filter can be understood as a function that:

- Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>).
- Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
    - If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
    -  If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [29]:
import re

def classify(message):
    """
    message: n string
    """
    
    message = re.sub("\W"," ", message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    if p_ham_given_message > p_spam_given_message:
        print("Label: Ham")
    elif p_ham_given_message < p_spam_given_message:
        print("Label: Spam")
    else:
        print("Equal Probablities, have a human classify this!")

In [30]:
classify("Winner!! This is secret code to unlock the money:1234")

Label: Spam


In [31]:
classify("Sounds good, Ebada, then see u there")

Label: Ham


## Measuring the Spam Filter's Accuracy

The two results above look promising, but let's see how well the filter does on our test set, which has 1,114 messages.

We'll start by writing a function that returns classification labels instead of printing them.

In [32]:
import re

def classify(message):
    """
    message: n string
    """
    
    message = re.sub("\W"," ", message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]
            
        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]
            
    if p_ham_given_message > p_spam_given_message:
        return "ham"
    elif p_ham_given_message < p_spam_given_message:
        return "spam"
    else:
        return "need human classification"

In [35]:
import pickle

pickle.dump(p_spam,open("p_spam.pkl","wb"))
pickle.dump(p_ham,open("p_ham.pkl","wb"))
pickle.dump(parameters_spam,open("parameters_spam.pkl","wb"))
pickle.dump(parameters_ham,open("parameters_ham.pkl","wb"))
pickle.dump(classify,open("classify","wb"))

Now that we have a function that returns labels instead of printing them, we can use it to create a new column in our test set.

In [33]:
test_set["predicted"] = test_set["SMS"].apply(classify)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now, we'll write a function to measure the accuracy of our spam filter to find out how well our spam filter does.

In [34]:
correct = 0
total = test_set.shape[0]

for index, row in test_set.iterrows():
    if row["Label"] == row["predicted"]:
        correct += 1
        
print("Correct: ", correct)
print("Incorrect: ", total - correct)
print("Accuracy: ", (correct/total) * 100)
    

Correct:  1100
Incorrect:  14
Accuracy:  98.74326750448833


The accuracy is close to 98.74%, which is really good. Our spam filter looked at 1,114 messages that it hasn't seen in training, and classified 1,100 correctly.